In [ ]:
!pip install tensorflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 104.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 110.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.1 MB/s eta 0:00:00


In [ ]:
import tensorflow as tf
print(tf.__version__)


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving combined_data_new2.dat to combined_data_new2.dat


In [ ]:
# Check the uploaded files
import os
print(os.listdir())

# Use the file in your code
file_path = 'combined_data_new2.dat'  # Replace with your file's name


['.config', 'combined_data_new2.dat', 'sample_data']


In [ ]:
import pickle

# Load the data from the uploaded file
file_path = 'combined_data_new2.dat'  # Replace with your file's name if different
with open(file_path, 'rb') as file:
    data_dict = pickle.load(file)

# Extract data and labels
data = data_dict['data']  # Shape: (num_samples, num_channels, num_time_points)
labels = data_dict['labels']  # Shape: (num_samples,)

# Check data shapes
num_samples, num_channels, num_time_points = data.shape
print(f"Number of samples: {num_samples}, Channels: {num_channels}, Time points: {num_time_points}")


Number of samples: 948, Channels: 64, Time points: 256


In [ ]:
import numpy as np
import tensorflow as tf
import scipy.signal as signal
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models
from tensorflow.keras import backend as K
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import pickle
import os
from sklearn.metrics import classification_report  # Import classification metrics

# Check the uploaded files (you should already know the file is uploaded)
print(os.listdir())  # This lists all files in the current directory

# Load the data from the uploaded file
file_path = 'combined_data_new2.dat'  # Direct file path after upload
with open(file_path, 'rb') as file:
    data_dict = pickle.load(file)

# Extract data and labels
data = data_dict['data']  # Shape: (num_samples, num_channels, num_time_points)
labels = data_dict['labels']  # Shape: (num_samples,)

num_samples, num_channels, num_time_points = data.shape
print(f"Number of samples: {num_samples}, Channels: {num_channels}, Time points: {num_time_points}")

# Bandpass Filter Function
def bandpass_filter(data, lowcut=1, highcut=50, fs=256, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = signal.butter(order, [low, high], btype='band')
    return signal.filtfilt(b, a, data)

# Power Spectral Density (PSD) computation function
def compute_psd(data, fs=256, nperseg=128):
    n_samples, n_channels, n_timepoints = data.shape
    psd_data = []
    for i in range(n_channels):
        channel_data = data[:, i, :]
        psd_per_channel = []
        for signal_data in channel_data:
            f, Pxx = signal.welch(signal_data, fs=fs, nperseg=nperseg)
            Pxx_log = np.log(Pxx + 1e-10)  # Avoid log(0)
            psd_per_channel.append(Pxx_log)
        psd_data.append(np.array(psd_per_channel))
    psd_data = np.array(psd_data)  # Shape: (n_channels, n_samples, n_freqs)
    return np.transpose(psd_data, (1, 0, 2))  # Shape: (n_samples, n_channels, n_freqs)

# Preprocess Data
filtered_data = np.zeros_like(data)
for i in range(num_channels):
    for j in range(num_samples):
        filtered_data[j, i, :] = bandpass_filter(data[j, i, :])  # Apply bandpass filter to each signal

# Compute PSD for filtered data
psd_data = compute_psd(filtered_data)

# Standardize the PSD data
num_samples, num_channels, num_freqs = psd_data.shape
psd_data = psd_data.reshape(-1, num_freqs)  # Flatten
scaler = StandardScaler()
psd_data = scaler.fit_transform(psd_data)
psd_data = psd_data.reshape(num_samples, num_channels, num_freqs)  # Reshape back

# PCA for dimensionality reduction (optional)
pca = PCA(n_components=20)  # Reduce to 20 components
psd_data_reduced = []
for channel in range(num_channels):
    channel_data = psd_data[:, channel, :]
    reduced_data = pca.fit_transform(channel_data)
    psd_data_reduced.append(reduced_data)

psd_data_reduced = np.stack(psd_data_reduced, axis=1)  # Shape: (num_samples, num_channels, 20)

# Compute class weights for imbalance handling
class_weights = compute_class_weight('balanced', classes=np.unique(labels), y=labels)
class_weight_dict = dict(enumerate(class_weights))

# Model training with entropy-based evaluation
accuracy_per_channel = []
precision_per_channel = []
recall_per_channel = []
f1_score_per_channel = []

for channel in range(num_channels):
    print(f"Training model for channel {channel + 1}/{num_channels}...")

    # Extract channel-specific data
    channel_data = psd_data_reduced[:, channel, :]  # Shape: (num_samples, n_features)

    # Split data into train-test sets
    X_train, X_test, y_train, y_test = train_test_split(channel_data, labels, test_size=0.1, random_state=42, stratify=labels)

    # Reshape for CNN input
    num_features = X_train.shape[1]
    X_train = X_train.reshape(-1, num_features, 1)  # Add 1D CNN channel dimension
    X_test = X_test.reshape(-1, num_features, 1)

    # Clear session
    K.clear_session()

    # Build the CNN model
    model = models.Sequential([
        layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(num_features, 1)),
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2),
        layers.Conv1D(128, kernel_size=3, activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling1D(pool_size=2),
        layers.Conv1D(256, kernel_size=3, activation='relu'),
        layers.BatchNormalization(),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),  # Prevent overfitting
        layers.Dense(1, activation='sigmoid')  # Binary classification
    ])

    # Compile the model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  loss='binary_crossentropy',  # Binary classification loss
                  metrics=['accuracy'])

    # Train the model
    model.fit(X_train, y_train, validation_split=0.1, epochs=30, batch_size=32, verbose=1,
              class_weight=class_weight_dict)

    # Evaluate on the test set
    y_pred = (model.predict(X_test) > 0.5).astype("int32")  # Get binary predictions (0 or 1)
    test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

    # Calculate Precision, Recall, F1-score
    report = classification_report(y_test, y_pred, output_dict=True)

    # Store metrics for each channel
    accuracy_per_channel.append((channel + 1, test_accuracy))
    precision_per_channel.append(report['macro avg']['precision'])
    recall_per_channel.append(report['macro avg']['recall'])
    f1_score_per_channel.append(report['macro avg']['f1-score'])

    print(f"Accuracy for channel {channel + 1}: {test_accuracy:.4f}")
    print(f"Precision for channel {channel + 1}: {report['macro avg']['precision']:.4f}")
    print(f"Recall for channel {channel + 1}: {report['macro avg']['recall']:.4f}")
    print(f"F1-score for channel {channel + 1}: {report['macro avg']['f1-score']:.4f}")

# Sort the channel-wise accuracies from largest to smallest
sorted_accuracies = sorted(zip(accuracy_per_channel, precision_per_channel, recall_per_channel, f1_score_per_channel), key=lambda x: x[0][1], reverse=True)

# Print channel-wise metrics
print("\nSorted Channel-wise Metrics:")
for (channel_info, precision, recall, f1) in sorted_accuracies:
    channel, accuracy = channel_info
    print(f"Channel {channel}: Accuracy = {accuracy:.4f}, Precision = {precision:.4f}, Recall = {recall:.4f}, F1-score = {f1:.4f}")

# Plot the accuracies for each channel
channels = [ch for ch, _ in sorted_accuracies]
accuracies = [acc for _, acc, _, _ in sorted_accuracies]

plt.figure(figsize=(10, 6))
plt.bar(channels, accuracies, color='skyblue')
plt.xlabel('Channel')
plt.ylabel('Accuracy')
plt.title('Channel-wise Accuracy (Using PSD Features Only)')
plt.show()


['.config', 'combined_data_new2.dat', 'sample_data']
Number of samples: 948, Channels: 64, Time points: 256
Training model for channel 1/64...
Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.4901 - loss: 1.0019 - val_accuracy: 0.5116 - val_loss: 0.6948
Epoch 2/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6026 - loss: 0.7605 - val_accuracy: 0.5000 - val_loss: 0.6920
Epoch 3/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6416 - loss: 0.6541 - val_accuracy: 0.5465 - val_loss: 0.6850
Epoch 4/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7389 - loss: 0.5486 - val_accuracy: 0.5930 - val_loss: 0.6806
Epoch 5/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7423 - loss: 0.5163 - val_accuracy: 0.5349 - val_loss: 0.6827
Epoch 6/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7280 - loss: 0.5389 - val_accuracy: 0.5116 - val_loss: 0.6860
Epoch 7/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7949 - loss: 0.4475 - val_accuracy: 0.5000 - val_loss: 0.7008
Epoch 8/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7921 - loss: 0.4461 - val_accuracy: 0.5000 - val_loss: 0.7190

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.4992 - loss: 0.9260 - val_accuracy: 0.5233 - val_loss: 0.6895
Epoch 2/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6190 - loss: 0.6837 - val_accuracy: 0.5349 - val_loss: 0.6862
Epoch 3/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6778 - loss: 0.6474 - val_accuracy: 0.6047 - val_loss: 0.6817
Epoch 4/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7611 - loss: 0.5178 - val_accuracy: 0.6047 - val_loss: 0.6766
Epoch 5/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7757 - loss: 0.4845 - val_accuracy: 0.5581 - val_loss: 0.6746
Epoch 6/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7518 - loss: 0.4780 - val_accuracy: 0.5349 - val_loss: 0.6731
Epoch 7/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8330 - loss: 0.3884 - val_accuracy: 0.5581 - val_loss: 0.6690
Epoch 8/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8380 - loss: 0.3835 - val_accuracy: 0.5698 - val_loss: 0.6668

...

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.5283 - loss: 0.8867 - val_accuracy: 0.5349 - val_loss: 0.6919
Epoch 2/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6035 - loss: 0.7159 - val_accuracy: 0.4884 - val_loss: 0.6928
Epoch 3/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6459 - loss: 0.6388 - val_accuracy: 0.4884 - val_loss: 0.6949
Epoch 4/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6781 - loss: 0.6285 - val_accuracy: 0.4884 - val_loss: 0.6985
Epoch 5/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7058 - loss: 0.5711 - val_accuracy: 0.4884 - val_loss: 0.7033
Epoch 6/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7127 - loss: 0.5477 - val_accuracy: 0.4884 - val_loss: 0.7004
Epoch 7/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7450 - loss: 0.4710 - val_accuracy: 0.4767 - val_loss: 0.7003
Epoch 8/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7863 - loss: 0.4492 - val_accuracy: 0.4884 - val_loss: 0.7024

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.5320 - loss: 0.9006 - val_accuracy: 0.5698 - val_loss: 0.6883
Epoch 2/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5986 - loss: 0.7623 - val_accuracy: 0.5116 - val_loss: 0.6864
Epoch 3/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6887 - loss: 0.6314 - val_accuracy: 0.5233 - val_loss: 0.6872
Epoch 4/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6982 - loss: 0.5996 - val_accuracy: 0.5116 - val_loss: 0.6886
Epoch 5/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7009 - loss: 0.5710 - val_accuracy: 0.5116 - val_loss: 0.6895
Epoch 6/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7482 - loss: 0.5400 - val_accuracy: 0.5233 - val_loss: 0.6867
Epoch 7/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7260 - loss: 0.5393 - val_accuracy: 0.5233 - val_loss: 0.6856
Epoch 8/30
24/24 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7881 - loss: 0.4481 - val_accuracy: 0.5233 - val_loss: 0.6858

ValueError: too many values to unpack (expected 2)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.utils import plot_model

# Assuming your model is already defined as 'model'
model = models.Sequential([
    layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(num_features, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(128, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(256, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  # Prevent overfitting
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

# Compile the model (use the same parameters you used earlier)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy',  # Binary classification loss
              metrics=['accuracy'])

# Visualize the model architecture
plot_model(model, to_file='model_architecture.png', show_shapes=True, show_layer_names=True)

# Optionally, display the image inline if using a Jupyter notebook
from IPython.display import Image
Image(filename='model_architecture.png')


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.9/319.9 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 112.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 109.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 115.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.9 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


In [ ]:
from tensorflow.keras.utils import plot_model
import shutil

# Define the CNN model architecture as you've described
model = models.Sequential([
    layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(num_features, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(128, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(256, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Visualize the model architecture and save it as an image
plot_model(model, to_file='/mnt/data/cnn_model_architecture.png', show_shapes=True, show_layer_names=True)

# Provide a download link to the user
from IPython.display import display, FileLink
display(FileLink('/mnt/data/cnn_model_architecture.png'))


You must install pydot (`pip install pydot`) for `plot_model` to work.


/mnt/data/cnn_model_architecture.png

In [ ]:
from tensorflow.keras.utils import plot_model
from google.colab import files

# Define the CNN model architecture as you've described
model = models.Sequential([
    layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(num_features, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(128, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(256, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Visualize the model architecture and save it as an image
model_plot_path = '/content/cnn_model_architecture.png'
plot_model(model, to_file=model_plot_path, show_shapes=True, show_layer_names=True)

# Initiate download for the model plot
files.download(model_plot_path)  # Download the model architecture plot

print("Download initiated for the model architecture plot.")


You must install pydot (`pip install pydot`) for `plot_model` to work.


FileNotFoundError: Cannot find file: /content/cnn_model_architecture.png

In [ ]:
from google.colab import drive
from tensorflow.keras.utils import plot_model

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define the CNN model architecture as you've described
model = models.Sequential([
    layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(num_features, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(128, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(256, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Step 3: Visualize the model architecture and save it to Google Drive
model_plot_path = '/content/drive/My Drive/Colab Files/Model Plots/cnn_model_architecture.png'
plot_model(model, to_file=model_plot_path, show_shapes=True, show_layer_names=True)

# Provide the path where the model plot is saved in Google Drive
print(f"Model architecture plot saved to Google Drive: {model_plot_path}")


Mounted at /content/drive
You must install pydot (`pip install pydot`) for `plot_model` to work.
Model architecture plot saved to Google Drive: /content/drive/My Drive/Colab Files/Model Plots/cnn_model_architecture.png


In [ ]:
!pip install pydot
!apt-get install -y graphviz


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
graphviz is already the newest version (2.42.2-6ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.


In [ ]:
from google.colab import drive
from tensorflow.keras.utils import plot_model

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define the CNN model architecture as you've described
model = models.Sequential([
    layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(num_features, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(128, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(256, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Step 3: Visualize the model architecture and save it to Google Drive
model_plot_path = '/content/drive/My Drive/Colab Files/Model Plots/cnn_model_architecture.png'
plot_model(model, to_file=model_plot_path, show_shapes=True, show_layer_names=True)

# Provide the path where the model plot is saved in Google Drive
print(f"Model architecture plot saved to Google Drive: {model_plot_path}")

# Download the model architecture plot
files.download(model_plot_path)  # Download the model architecture plot

print("Download initiated for the model architecture plot.")

ValueError: mount failed

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
drive.flush_and_unmount()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
from google.colab import drive, files
from tensorflow.keras.utils import plot_model
from tensorflow.keras import models, layers

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define the CNN model architecture as you've described
model = models.Sequential([
    layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(num_features, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(128, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(256, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Step 3: Visualize the model architecture and save it to Google Drive
model_plot_path = '/content/drive/My Drive/Colab Files/Model Plots/cnn_model_architecture.png'
plot_model(model, to_file=model_plot_path, show_shapes=True, show_layer_names=True)

# Provide the path where the model plot is saved in Google Drive
print(f"Model architecture plot saved to Google Drive: {model_plot_path}")

# Step 4: Move the plot from Google Drive to local `/content/` directory for download
local_path = '/content/cnn_model_architecture.png'
!cp "{model_plot_path}" "{local_path}"

# Step 5: Download the model architecture plot
files.download(local_path)  # Download the model architecture plot

print("Download initiated for the model architecture plot.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


NameError: name 'num_features' is not defined

In [ ]:
from google.colab import drive, files
from tensorflow.keras.utils import plot_model
from tensorflow.keras import models, layers

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define the number of features
# Assuming psd_data_reduced is the final input for the model and has been reduced to 20 features via PCA
num_features = 20  # As we used PCA(n_components=20)

# Define the CNN model architecture
model = models.Sequential([
    layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(num_features, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(128, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(256, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Step 3: Visualize the model architecture and save it to Google Drive
model_plot_path = '/content/drive/My Drive/Colab Files/Model Plots/cnn_model_architecture.png'
plot_model(model, to_file=model_plot_path, show_shapes=True, show_layer_names=True)

# Provide the path where the model plot is saved in Google Drive
print(f"Model architecture plot saved to Google Drive: {model_plot_path}")

# Step 4: Move the plot from Google Drive to local `/content/` directory for download
local_path = '/content/cnn_model_architecture.png'
!cp "{model_plot_path}" "{local_path}"

# Step 5: Download the model architecture plot
files.download(local_path)  # Download the model architecture plot

print("Download initiated for the model architecture plot.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
You must install pydot (`pip install pydot`) for `plot_model` to work.
Model architecture plot saved to Google Drive: /content/drive/My Drive/Colab Files/Model Plots/cnn_model_architecture.png


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


cp: cannot stat '/content/drive/My Drive/Colab Files/Model Plots/cnn_model_architecture.png': No such file or directory


FileNotFoundError: Cannot find file: /content/cnn_model_architecture.png

In [ ]:
!pip install pydot
!apt-get install -y graphviz


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
graphviz is already the newest version (2.42.2-6ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.


In [ ]:
from google.colab import drive, files
from tensorflow.keras.utils import plot_model
from tensorflow.keras import models, layers

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define the number of features
# Assuming psd_data_reduced is the final input for the model and has been reduced to 20 features via PCA
num_features = 20  # As we used PCA(n_components=20)

# Define the CNN model architecture
model = models.Sequential([
    layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(num_features, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(128, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(256, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Step 3: Visualize the model architecture and save it to Google Drive
model_plot_path = '/content/drive/My Drive/Colab Files/Model Plots/cnn_model_architecture.png'
plot_model(model, to_file=model_plot_path, show_shapes=True, show_layer_names=True)

# Provide the path where the model plot is saved in Google Drive
print(f"Model architecture plot saved to Google Drive: {model_plot_path}")

# Step 4: Move the plot from Google Drive to local `/content/` directory for download
local_path = '/content/cnn_model_architecture.png'
!cp "{model_plot_path}" "{local_path}"

# Step 5: Download the model architecture plot
files.download(local_path)  # Download the model architecture plot

print("Download initiated for the model architecture plot.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
You must install pydot (`pip install pydot`) for `plot_model` to work.
Model architecture plot saved to Google Drive: /content/drive/My Drive/Colab Files/Model Plots/cnn_model_architecture.png
cp: cannot stat '/content/drive/My Drive/Colab Files/Model Plots/cnn_model_architecture.png': No such file or directory


FileNotFoundError: Cannot find file: /content/cnn_model_architecture.png

In [ ]:
from google.colab import drive, files
from tensorflow.keras.utils import plot_model
from tensorflow.keras import models, layers

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define the number of features
num_features = 20  # As we used PCA(n_components=20)

# Define the CNN model architecture
model = models.Sequential([
    layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(num_features, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(128, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(256, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Step 3: Visualize the model architecture and save it to Google Drive
model_plot_path = '/content/drive/My Drive/Colab Files/Model Plots/cnn_model_architecture.png'
plot_model(model, to_file=model_plot_path, show_shapes=True, show_layer_names=True)

# Provide the path where the model plot is saved in Google Drive
print(f"Model architecture plot saved to Google Drive: {model_plot_path}")

# Step 4: Directly use Google Drive's path to download the file
# If you want to download the file directly from Google Drive, use this link:
print(f"Direct link to download the model architecture plot: {model_plot_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
You must install pydot (`pip install pydot`) for `plot_model` to work.
Model architecture plot saved to Google Drive: /content/drive/My Drive/Colab Files/Model Plots/cnn_model_architecture.png
Direct link to download the model architecture plot: /content/drive/My Drive/Colab Files/Model Plots/cnn_model_architecture.png


In [ ]:
from google.colab import drive
from tensorflow.keras.utils import plot_model
from tensorflow.keras import models, layers

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define the number of features
num_features = 20  # As we used PCA(n_components=20)

# Define the CNN model architecture
model = models.Sequential([
    layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(num_features, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(128, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(256, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Step 3: Define the new save path
model_plot_path = '/content/drive/My Drive/cnn_model_architecture.png'

# Step 4: Visualize the model architecture and save it to Google Drive
plot_model(model, to_file=model_plot_path, show_shapes=True, show_layer_names=True)

# Provide the path where the model plot is saved in Google Drive
print(f"Model architecture plot saved to Google Drive: {model_plot_path}")

# Check if the file is saved in Google Drive and provide the link


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
You must install pydot (`pip install pydot`) for `plot_model` to work.
Model architecture plot saved to Google Drive: /content/drive/My Drive/cnn_model_architecture.png


In [ ]:
from google.colab import files

# Path to the saved histogram plot in Google Drive

model_plot_path = '/content/drive/My Drive/cnn_model_architecture.png'

# Download the histogram plot
files.download(model_plot_path)  # Download the histogram plot

print("Download initiated for the model_plot_path.")

FileNotFoundError: Cannot find file: /content/drive/My Drive/cnn_model_architecture.png

In [ ]:
from google.colab import files
import shutil

# Path to the saved model plot in Google Drive
model_plot_path = '/content/drive/My Drive/cnn_model_architecture.png'

# Local path to save the file temporarily for download
local_path = '/content/cnn_model_architecture.png'

# Copy the file from Google Drive to local
try:
    shutil.copy(model_plot_path, local_path)
    print(f"File copied successfully from {model_plot_path} to {local_path}")

    # Download the model architecture plot
    files.download(local_path)  # Download the model architecture plot
    print("Download initiated for the model architecture plot.")
except FileNotFoundError:
    print(f"Error: The file at {model_plot_path} was not found. Please verify the path.")


Error: The file at /content/drive/My Drive/cnn_model_architecture.png was not found. Please verify the path.


In [ ]:
from google.colab import drive
from tensorflow.keras.utils import plot_model
from tensorflow.keras import models, layers

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define the number of features
num_features = 20  # As we used PCA(n_components=20)

# Define the CNN model architecture
model = models.Sequential([
    layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(num_features, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(128, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(256, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Step 3: Define the new save path
model_plot_path = '/content/drive/My Drive/cnn_model_architecture.png'  # Or modify this path

# Step 4: Visualize the model architecture and save it to Google Drive
plot_model(model, to_file=model_plot_path, show_shapes=True, show_layer_names=True)

# Step 5: Provide the path where the model plot is saved in Google Drive
print(f"Model architecture plot saved to Google Drive: {model_plot_path}")

# Check the file in Google Drive manually by opening this link:
print(f"Check the saved file in Google Drive: {model_plot_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
You must install pydot (`pip install pydot`) for `plot_model` to work.
Model architecture plot saved to Google Drive: /content/drive/My Drive/cnn_model_architecture.png
Check the saved file in Google Drive: /content/drive/My Drive/cnn_model_architecture.png


In [ ]:
from google.colab import drive
from tensorflow.keras.utils import plot_model
from tensorflow.keras import models, layers

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define the number of features
num_features = 20  # As we used PCA(n_components=20)

# Define the CNN model architecture
model = models.Sequential([
    layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(num_features, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(128, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(256, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Step 3: Save the model architecture plot to the root of My Drive
model_plot_path ='/content/drive/My Drive/Colab Files/Model Plots/cnn_model_architecture.png'


# Step 4: Visualize the model architecture and save it to Google Drive
plot_model(model, to_file=model_plot_path, show_shapes=True, show_layer_names=True)

# Step 5: Provide the path where the model plot is saved in Google Drive
print(f"Model architecture plot saved to Google Drive: {model_plot_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
You must install pydot (`pip install pydot`) for `plot_model` to work.
Model architecture plot saved to Google Drive: /content/drive/My Drive/Colab Files/Model Plots/cnn_model_architecture.png


In [ ]:
from google.colab import files

# Path to the saved histogram plot in Google Drive
 model_plot_path = '/content/drive/My Drive/Colab Files/Model Plots/cnn_model_architecture.png'

# Download the histogram plot
files.download(model_plot_path)  # Download the histogram plot

print("Download initiated for model_plot_path.")


IndentationError: unexpected indent (ipython-input-23-2952261693.py, line 4)

In [ ]:

!pip install pydot
!apt-get install -y graphviz


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
graphviz is already the newest version (2.42.2-6ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.


In [ ]:
from google.colab import drive
from tensorflow.keras.utils import plot_model
from tensorflow.keras import models, layers

# Step 1: Mount Google Drive
drive.mount('/content/drive')

# Step 2: Define the number of features
num_features = 20  # As we used PCA(n_components=20)

# Define the CNN model architecture
model = models.Sequential([
    layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(num_features, 1)),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(128, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(256, kernel_size=3, activation='relu'),
    layers.BatchNormalization(),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Step 3: Save the model plot to the root of My Drive (simplified path)
model_plot_path = '/content/drive/My Drive/cnn_model_architecture.png'

# Step 4: Visualize the model architecture and save it to Google Drive
plot_model(model, to_file=model_plot_path, show_shapes=True, show_layer_names=True)

# Step 5: Provide the path where the model plot is saved in Google Drive
print(f"Model architecture plot saved to Google Drive: {model_plot_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
You must install pydot (`pip install pydot`) for `plot_model` to work.
Model architecture plot saved to Google Drive: /content/drive/My Drive/cnn_model_architecture.png


In [ ]:
model_plot_path = '/content/drive/My Drive/Colab Files/Model Plots/cnn_model_architecture.png'
